### Find variable sites from alignments
Given an alignment with a reference sequence as the first sequence, find all variable sites and return a list of these sites.

In [ ]:
from Bio import SeqIO
import pandas as pd
from collections import Counter

In [ ]:
# Function to analyze variable sites in a FASTA alignment file
def analyze_variable_sites_detailed(fasta_file, min_differences=2):
    """
    Analyze variable sites in a multiple sequence alignment FASTA file.
    The first sequence in the FASTA file is treated as the reference (wildtype).
    Returns a DataFrame with detailed mutation information for sites with at least
    `min_differences` mutations.
    """
    sequences = list(SeqIO.parse(fasta_file, "fasta"))
    ref_seq = str(sequences[0].seq)
    seq_length = len(ref_seq)

    results = []

    for pos in range(seq_length):
        ref_aa = ref_seq[pos]
        mutant_aas = []

        for seq in sequences[1:]:
            current_aa = str(seq.seq)[pos]
            if current_aa != ref_aa:
                mutant_aas.append(current_aa)

        # Count frequency of each mutant amino acid at this position
        if mutant_aas:  # If there are any mutations at this site
            aa_counts = Counter(mutant_aas)

            # Add a row for each mutation type that meets the threshold
            for mutant_aa, count in aa_counts.items():
                if count >= min_differences:
                    results.append(
                        {
                            "site": pos + 1,  # 1-based positioning
                            "wildtype": ref_aa,
                            "mutant": mutant_aa,
                            "mutation_count": count,
                            "mutation_type": f"{ref_aa}->{mutant_aa}",
                        }
                    )

    # Create DataFrame and sort by site, then by mutation count (descending)
    df = pd.DataFrame(results)
    if not df.empty:
        df = df.sort_values(["site", "mutation_count"], ascending=[True, False])
        df = df.reset_index(drop=True)

    return df


# Usage
hendra = analyze_variable_sites_detailed(snakemake.input.hendra_F_alignment)
display(hendra)
hendra.to_csv(snakemake.output.hendra_F_variable_sites, index=False)

nipah = analyze_variable_sites_detailed(snakemake.input.nipah_F_alignment)
display(nipah)
nipah.to_csv(snakemake.output.nipah_F_variable_sites, index=False)
